In [3]:
import torchfile
import numpy as np
import time, sys
import os

from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Convolution2D, Dense, Flatten, Activation, MaxPooling2D, Permute
from keras.constraints import nonneg
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


# Load Data

In [4]:
train_data = torchfile.load('/Users/jaichitra.balakrishnan/galvanizeu/DSCI6005-FinalProject/data/trainobjcat.t7')
test_data = torchfile.load('/Users/jaichitra.balakrishnan/galvanizeu/DSCI6005-FinalProject/data/valobjcat.t7')

# Test and Train Data

In [5]:
X_train = train_data[b'x'][0]
y_train = train_data[b'y']
X_test = test_data[b'x'][0]
y_test = test_data[b'y']

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6415, 3, 128, 128), (6415,), (1604, 3, 128, 128), (1604,))

In [11]:
unique, counts = np.unique(np.concatenate((y_train,y_test), axis=0), return_counts=True)

In [14]:
unique, counts
print (np.asarray((unique, counts)).T)

[[ 26  12]
 [ 27  30]
 [ 28  30]
 [ 29  36]
 [ 30  40]
 [ 31  30]
 [ 32  20]
 [ 33  28]
 [ 34  23]
 [ 35  30]
 [ 36  29]
 [ 37  27]
 [ 38  34]
 [ 39  36]
 [ 40  36]
 [ 41  29]
 [ 42  27]
 [ 43  25]
 [ 44  25]
 [ 45  32]
 [ 46  34]
 [ 47  29]
 [ 48  44]
 [ 49  36]
 [ 50  24]
 [ 51  25]
 [ 52  27]
 [ 53  21]
 [ 54  30]
 [ 55  35]
 [ 56  14]
 [ 57  26]
 [ 58  26]
 [ 59  27]
 [ 60  25]
 [ 61   9]
 [ 62   8]
 [ 63  12]
 [ 64  21]
 [ 65  27]
 [ 66  25]
 [ 67  41]
 [ 68  21]
 [ 69  28]
 [ 70  23]
 [ 71  37]
 [ 72  46]
 [ 73  76]
 [ 74  73]
 [ 75  34]
 [ 76  29]
 [ 77  25]
 [ 78  26]
 [ 79  24]
 [ 80  23]
 [ 81  23]
 [ 82  28]
 [ 83  35]
 [ 84  36]
 [ 85  41]
 [ 86  16]
 [ 87  16]
 [ 88  18]
 [ 89  52]
 [ 90  25]
 [ 91  26]
 [ 92  28]
 [ 93  23]
 [ 94  46]
 [ 95  25]
 [ 96  27]
 [ 97  26]
 [ 98  26]
 [ 99  25]
 [100  21]
 [101  36]
 [102  28]
 [103  35]
 [104  21]
 [105  23]
 [106  44]
 [107  25]
 [108  26]
 [109  38]
 [110  37]
 [111  36]
 [112  35]
 [113  34]
 [114  29]
 [115  34]
 [116  38]

# Model Architecture

In [25]:
nb_filter = 2000
model = Sequential()
model.add(Convolution2D(filters=nb_filter, data_format = 'channels_first', kernel_size=10, kernel_constraint=nonneg(), kernel_regularizer=l2(1e-3), name='conv', input_shape=X_train[0].shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(10, 10)))
model.add(Permute([3, 1, 2]))

model.add(Convolution2D(filters=nb_filter, data_format = 'channels_first', kernel_size=10, kernel_constraint=nonneg(), kernel_regularizer=l2(1e-3), name='conv1', input_shape=X_train[0].shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(10, 10)))

model.add(Convolution2D(filters=nb_filter, data_format = 'channels_first', kernel_size=2, kernel_constraint=nonneg(), kernel_regularizer=l2(1e-3), name='conv2', input_shape=X_train[0].shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten(name='flatten'))

model.add(Dense(units=282, kernel_regularizer=l2(1e-3), name='softmax_weights'))
model.add(Activation('softmax', name='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv (Conv2D)                (None, 2000, 119, 119)    602000    
_________________________________________________________________
activation_35 (Activation)   (None, 2000, 119, 119)    0         
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 200, 11, 119)      0         
_________________________________________________________________
permute_17 (Permute)         (None, 119, 200, 11)      0         
_________________________________________________________________
conv1 (Conv2D)               (None, 2000, 191, 2)      23802000  
_________________________________________________________________
activation_36 (Activation)   (None, 2000, 191, 2)      0         
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 200, 19, 2)        0         
__________

# Fit Model

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
nb_train = len(y_train)
model.fit(X_train[:nb_train], to_categorical(y_train[:nb_train]), validation_split=.1, epochs=10)

Train on 5773 samples, validate on 642 samples
Epoch 1/10
 128/5773 [..............................] - ETA: 32874s - loss: 14.1124 - acc: 0.0078

In [3]:
DATA_DIR = os.getcwd().rsplit('/', 1)[0] + '/data'
checkpoint_dir = os.getcwd().rsplit('/', 1)[0] + '/checkpoint'